In [1]:
from datasets import load_dataset, Dataset, load_from_disk
from tqdm import tqdm, trange
import json
from __future__ import annotations
from typing import Iterator
import torch
from torch.utils.data import BatchSampler, DataLoader
from torch import nn, Tensor
from typing import Dict, Type, Callable, List
from torch.optim import Optimizer
import os
from __future__ import annotations

import csv
import os
from contextlib import nullcontext
from typing import TYPE_CHECKING

import numpy as np
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances

from sentence_transformers.evaluation.SentenceEvaluator import SentenceEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers.similarity_functions import SimilarityFunction
from sentence_transformers.SentenceTransformer import SentenceTransformer
import gzip

### Using an off the shelf open source dataset here. You can replace with your own dataset (Transformers Dataset Format) with columns,
### Anchor, Positive, Negative in that sequence

In [2]:
# Format Data with columns names: 'anchor', 'positive', 'negative'

In [2]:
# Replace this with your train, eval, test datasets

dataset = load_dataset("tomaarsen/gooaq-hard-negatives", "triplet-all", split="train", streaming=True, cache_dir="D:/temp")

number_samples = 7000
count = 0
train_data_samples = []
valid_data_samples = []
test_data_samples = []

for i in tqdm(dataset):
    if count >= 5000 and count < 6000:
        valid_data_samples.append(i)
    if count >= 6000:
        test_data_samples.append(i)
    if count < 5000:
        train_data_samples.append(i)
    count += 1
    if count == number_samples:
        break

train_data = Dataset.from_list(train_data_samples)
valid_data = Dataset.from_list(valid_data_samples)
test_data = Dataset.from_list(test_data_samples)

6999it [00:01, 5566.00it/s]


In [2]:
train_data = load_from_disk("D:/embeddings_gooaq_subset/train")
valid_data = load_from_disk("D:/embeddings_gooaq_subset/valid")
test_data = load_from_disk("D:/embeddings_gooaq_subset/test")

In [3]:
len(train_data), len(valid_data), len(test_data)

(88194, 4924, 4893)

In [4]:
def get_unique_texts(train_data, valid_data=None, test_data=None):
    unique_texts = set()
    
    unique_texts.update(train_data['anchor'])
    unique_texts.update(train_data['positive'])
    unique_texts.update(train_data['negative'])

    if valid_data is not None:
        unique_texts.update(valid_data['anchor'])
        unique_texts.update(valid_data['positive'])
        unique_texts.update(valid_data['negative'])

    if test_data is not None:
        unique_texts.update(test_data['anchor'])
        unique_texts.update(test_data['positive'])
        unique_texts.update(test_data['negative'])
    
    return list(unique_texts)

In [5]:
unique_texts = get_unique_texts(train_data, valid_data, test_data)

In [6]:
len(unique_texts)

113767

### Loading an Open Source Model Here to create Embedding, Replace this with a Closed source model endpoint to get the embeddings

In [7]:
from sentence_transformers import SentenceTransformer
sentences = ["Hello", "I am Osama"]
model = SentenceTransformer('tomaarsen/mpnet-base-nli-matryoshka')

In [8]:
embeddings_1 = model.encode(sentences, normalize_embeddings=True)
embeddings_1

array([[-0.03840512, -0.02365682, -0.01091154, ...,  0.05426791,
        -0.01852449, -0.03838428],
       [-0.02164984, -0.01609711,  0.01288171, ..., -0.01077557,
        -0.02442665,  0.02066883]], dtype=float32)

In [9]:
# We get the embeddings of all the texts and save them. This is so that we dont compute them everytime from the closed source model, when we are training
dict_embeddings = {}
batch_size = 1000

for batch in tqdm(range(0, len(unique_texts), batch_size)):
    list_embeddings = model.encode(unique_texts[batch : batch + batch_size])
    for idx, u_t in enumerate(unique_texts[batch : batch + batch_size]):
        dict_embeddings[u_t] = list_embeddings[idx].tolist()

100%|██████████| 114/114 [09:47<00:00,  5.16s/it]


In [16]:
output_file = 'D:/closed_model_embeddings.json.gz'

# Save as compressed JSON
with gzip.open(output_file, 'wt', encoding='utf-8') as f:
    json.dump(dict_embeddings, f)

In [ ]:
with gzip.open('D:/closed_model_embeddings.json.gz', 'rt', encoding='utf-8') as f:
    dict_embeddings = json.load(f)

In [10]:
len(dict_embeddings)

113767

### Dataset Formatting

In [11]:
class SetEpochMixin:
    """
    Required for a BatchSampler as the Trainer will call set_epoch on the BatchSampler at the beginning of each epoch.
    The BatchSampler can then set the generator seed accordingly.
    """

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.epoch = 0

    def set_epoch(self, epoch: int) -> None:
        self.epoch = epoch

class NoDuplicatesBatchSampler(SetEpochMixin, BatchSampler):
    def __init__(
        self,
        dataset: Dataset,
        batch_size: int,
        drop_last: bool,
        valid_label_columns: list[str] = [],
        generator: torch.Generator = None,
        seed: int = 0,
    ) -> None:
        """
        This sampler creates batches such that each batch contains samples where the values are unique,
        even across columns. This is useful when losses consider other samples in a batch to be in-batch
        negatives, and you want to ensure that the negatives are not duplicates of the anchor/positive sample.

        Recommended for:
            - :class:`~sentence_transformers.losses.MultipleNegativesRankingLoss`
            - :class:`~sentence_transformers.losses.CachedMultipleNegativesRankingLoss`
            - :class:`~sentence_transformers.losses.MultipleNegativesSymmetricRankingLoss`
            - :class:`~sentence_transformers.losses.CachedMultipleNegativesSymmetricRankingLoss`
            - :class:`~sentence_transformers.losses.MegaBatchMarginLoss`
            - :class:`~sentence_transformers.losses.GISTEmbedLoss`
            - :class:`~sentence_transformers.losses.CachedGISTEmbedLoss`

        Args:
            dataset (Dataset): The dataset to sample from.
            batch_size (int): Number of samples per batch.
            drop_last (bool): If True, drop the last incomplete batch if the dataset size
                is not divisible by the batch size.
            valid_label_columns (List[str]): List of column names to check for labels.
                The first column name from ``valid_label_columns`` found in the dataset will
                be used as the label column.
            generator (torch.Generator, optional): Optional random number generator for shuffling
                the indices.
            seed (int, optional): Seed for the random number generator to ensure reproducibility.
        """
        super().__init__(dataset, batch_size, drop_last)
        if label_columns := set(dataset.column_names) & (set(valid_label_columns) | {"dataset_name"}):
            dataset = dataset.remove_columns(label_columns)
        self.dataset = dataset
        self.batch_size = batch_size
        self.drop_last = drop_last
        self.generator = generator
        self.seed = seed

    def __iter__(self) -> Iterator[list[int]]:
        """
        Iterate over the remaining non-yielded indices. For each index, check if the sample values are already in the
        batch. If not, add the sample values to the batch keep going until the batch is full. If the batch is full, yield
        the batch indices and continue with the next batch.
        """
        if self.generator and self.seed:
            self.generator.manual_seed(self.seed + self.epoch)

        remaining_indices = set(torch.randperm(len(self.dataset), generator=self.generator).tolist())
        while remaining_indices:
            batch_values = set()
            batch_indices = []
            for index in remaining_indices:
                sample_values = set(self.dataset[index].values())
                if sample_values & batch_values:
                    continue

                batch_indices.append(index)
                if len(batch_indices) == self.batch_size:
                    yield batch_indices
                    break

                batch_values.update(sample_values)

            else:
                # NOTE: some indices might still have been ignored here
                if not self.drop_last:
                    yield batch_indices

            remaining_indices -= set(batch_indices)

    def __len__(self) -> int:
        if self.drop_last:
            return len(self.dataset) // self.batch_size
        else:
            return (len(self.dataset) + self.batch_size - 1) // self.batch_size

In [12]:
sampler = NoDuplicatesBatchSampler(train_data, 100, False, generator=torch.Generator())
dataloader_params = {}
dataloader_params["batch_sampler"] = sampler
train_dl = DataLoader(train_data, **dataloader_params)

In [13]:
# Sample from dataloader
for d in train_dl:
    break

d

{'anchor': ['is toprol xl the same as metoprolol?',
  'are you experienced cd steve hoffman?',
  'how are babushka dolls made?',
  'are eyes always the same size?',
  'how long do you have to wait to apply for cerb?',
  'what is the volkswagen commercial song?',
  'what does it mean when it says your internet connection is unstable?',
  '3 what does this symbol mean?',
  'what is the difference between joint ligament and tendon?',
  'are equilateral triangles always equiangular?',
  'how to fix an error occurred while reconnecting z?',
  'do the alps always have snow?',
  'is focalin safer than adderall?',
  '6 mg is how many milliliters?',
  'are pyle tvs any good?',
  'how many moles of co2 are produced when 5.45 mol of ethane are burned?',
  'what are the requirements to become a health inspector?',
  'ring is turning finger black?',
  'can not drinking enough water cause preterm labor?',
  'are bent over rows safe?',
  'is himalayan pink salt dyed?',
  'is corn silk used for?',
  '

### Loading / Creating Adapter Model

In [14]:
class AdapterModel(nn.Module):
    def __init__(self, dim: int, num_layers: int):
        super(AdapterModel, self).__init__()
        self.layers = nn.ModuleList()
        
        for _ in range(num_layers):
            self.layers.append(nn.Linear(dim, dim))  # Each layer has the same dimension
            
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [15]:
# We get the embedding dimension and create linear layers of the same dimension
# For a closed source model you can check the length of the embedding dimension
embedding_dimension = 768

adapter_model = AdapterModel(embedding_dimension, 1)

In [16]:
adapter_model

AdapterModel(
  (layers): ModuleList(
    (0): Linear(in_features=768, out_features=768, bias=True)
  )
)

### If model is saved at a location, and you want to load

In [17]:
model_load_location = "D:/adapter_model/checkpoints/model_checkpoint_300.pth"

try:
    adapter_model.load_state_dict(torch.load(model_load_location, weights_only=True)['model_state_dict'])
    adapter_model.eval()
except Exception as e:
    print(f"ERROR: {str(e)}")

ERROR: [Errno 2] No such file or directory: 'D:/adapter_model/checkpoints/model_checkpoint_300.pth'


### Evaluation Code

In [18]:
class TripletEvaluator(SentenceEvaluator):
    """
    Evaluate a model based on a triplet: (sentence, positive_example, negative_example).
    Checks if distance(sentence, positive_example) < distance(sentence, negative_example).

    Example:
        ::

            from sentence_transformers import SentenceTransformer
            from sentence_transformers.evaluation import TripletEvaluator
            from datasets import load_dataset

            # Load a model
            model = SentenceTransformer('all-mpnet-base-v2')

            # Load a dataset with (anchor, positive, negative) triplets
            dataset = load_dataset("sentence-transformers/all-nli", "triplet", split="dev")

            # Initialize the TripletEvaluator using anchors, positives, and negatives
            triplet_evaluator = TripletEvaluator(
                anchors=dataset[:1000]["anchor"],
                positives=dataset[:1000]["positive"],
                negatives=dataset[:1000]["negative"],
                name="all-nli-dev",
            )
            results = triplet_evaluator(model)
            '''
            TripletEvaluator: Evaluating the model on the all-nli-dev dataset:
            Accuracy Cosine Distance:        95.60
            Accuracy Dot Product:            4.40
            Accuracy Manhattan Distance:     95.40
            Accuracy Euclidean Distance:     95.60
            '''
            print(triplet_evaluator.primary_metric)
            # => "all-nli-dev_max_accuracy"
            print(results[triplet_evaluator.primary_metric])
            # => 0.956
    """

    def __init__(
        self,
        anchors: list[str],
        positives: list[str],
        negatives: list[str],
        main_distance_function: str | SimilarityFunction | None = None,
        name: str = "",
        batch_size: int = 16,
        write_csv: bool = True,
        truncate_dim: int | None = None,
    ):
        """
        Initializes a TripletEvaluator object.

        Args:
            anchors (List[str]): Sentences to check similarity to. (e.g. a query)
            positives (List[str]): List of positive sentences
            negatives (List[str]): List of negative sentences
            main_distance_function (Union[str, SimilarityFunction], optional):
                The distance function to use. If not specified, use cosine similarity,
                dot product, Euclidean, and Manhattan. Defaults to None.
            name (str): Name for the output. Defaults to "".
            batch_size (int): Batch size used to compute embeddings. Defaults to 16.
            write_csv (bool): Write results to a CSV file. Defaults to True.
            truncate_dim (int, optional): The dimension to truncate sentence embeddings to.
                `None` uses the model's current truncation dimension. Defaults to None.
        """
        super().__init__()
        self.anchors = anchors
        self.positives = positives
        self.negatives = negatives
        self.name = name
        self.truncate_dim = truncate_dim

        assert len(self.anchors) == len(self.positives)
        assert len(self.anchors) == len(self.negatives)

        self.main_distance_function = SimilarityFunction(main_distance_function) if main_distance_function else None

        self.batch_size = batch_size

        self.csv_file: str = "triplet_evaluation" + ("_" + name if name else "") + "_results.csv"
        self.csv_headers = ["epoch", "steps", "accuracy_cosine", "accuracy_manhattan", "accuracy_euclidean"]
        self.write_csv = write_csv

    @classmethod
    def from_input_examples(cls, examples: list[InputExample], **kwargs):
        anchors = []
        positives = []
        negatives = []

        for example in examples:
            anchors.append(example.texts[0])
            positives.append(example.texts[1])
            negatives.append(example.texts[2])
        return cls(anchors, positives, negatives, **kwargs)

    def __call__(
        self, model, closed_source: bool, output_path: str = None, epoch: int = -1, steps: int = -1
    ) -> dict[str, float]:
        if epoch != -1:
            if steps == -1:
                out_txt = f" after epoch {epoch}"
            else:
                out_txt = f" in epoch {epoch} after {steps} steps"
        else:
            out_txt = ""
        if self.truncate_dim is not None:
            out_txt += f" (truncated to {self.truncate_dim})"

        print(f"TripletEvaluator: Evaluating the model on the {self.name} dataset{out_txt}:")

        num_triplets = 0
        (
            num_correct_cos_triplets,
            num_correct_dot_triplets,
            num_correct_manhattan_triplets,
            num_correct_euclidean_triplets,
        ) = 0, 0, 0, 0

        with nullcontext() if self.truncate_dim is None else model.truncate_sentence_embeddings(self.truncate_dim):
            
            embedding_anchors_closed = [dict_embeddings[i] for i in self.anchors]
            embedding_positives_closed = [dict_embeddings[i] for i in self.positives]
            embedding_negatives_closed = [dict_embeddings[i] for i in self.negatives]

            if not closed_source:
                embeddings_anchors = []
                for b in trange(0, len(self.anchors), self.batch_size):
                    with torch.no_grad():
                        embeddings_anchors += model(torch.tensor(embedding_anchors_closed[b : b + self.batch_size])).tolist()
                embeddings_anchors = np.array(embeddings_anchors)

                embeddings_positives = []
                for b in trange(0, len(self.positives), self.batch_size):
                    with torch.no_grad():
                        embeddings_positives += model(torch.tensor(embedding_positives_closed[b : b + self.batch_size])).tolist()
                embeddings_positives = np.array(embeddings_positives)

                embeddings_negatives = []
                for b in trange(0, len(self.negatives), self.batch_size):
                    with torch.no_grad():
                        embeddings_negatives += model(torch.tensor(embedding_negatives_closed[b : b + self.batch_size])).tolist()
                embeddings_negatives = np.array(embeddings_negatives)

            else:
                embeddings_anchors = np.array(embedding_anchors_closed)
                embeddings_positives = np.array(embedding_positives_closed)
                embeddings_negatives = np.array(embedding_negatives_closed)

        # Cosine distance
        pos_cos_distance = paired_cosine_distances(embeddings_anchors, embeddings_positives)
        neg_cos_distances = paired_cosine_distances(embeddings_anchors, embeddings_negatives)

        # Dot score
        pos_dot_distance = np.sum(embeddings_anchors * embeddings_positives, axis=-1)
        neg_dot_distances = np.sum(embeddings_anchors * embeddings_negatives, axis=-1)

        # Manhattan
        pos_manhattan_distance = paired_manhattan_distances(embeddings_anchors, embeddings_positives)
        neg_manhattan_distances = paired_manhattan_distances(embeddings_anchors, embeddings_negatives)

        # Euclidean
        pos_euclidean_distance = paired_euclidean_distances(embeddings_anchors, embeddings_positives)
        neg_euclidean_distances = paired_euclidean_distances(embeddings_anchors, embeddings_negatives)

        for idx in range(len(pos_cos_distance)):
            num_triplets += 1

            if pos_cos_distance[idx] < neg_cos_distances[idx]:
                num_correct_cos_triplets += 1

            if pos_dot_distance[idx] < neg_dot_distances[idx]:
                num_correct_dot_triplets += 1

            if pos_manhattan_distance[idx] < neg_manhattan_distances[idx]:
                num_correct_manhattan_triplets += 1

            if pos_euclidean_distance[idx] < neg_euclidean_distances[idx]:
                num_correct_euclidean_triplets += 1

        accuracy_cos = num_correct_cos_triplets / num_triplets
        accuracy_dot = num_correct_dot_triplets / num_triplets
        accuracy_manhattan = num_correct_manhattan_triplets / num_triplets
        accuracy_euclidean = num_correct_euclidean_triplets / num_triplets

        print(f"Accuracy Cosine Distance:   \t{accuracy_cos * 100:.2f}")
        print(f"Accuracy Dot Product:       \t{accuracy_dot * 100:.2f}")
        print(f"Accuracy Manhattan Distance:\t{accuracy_manhattan * 100:.2f}")
        print(f"Accuracy Euclidean Distance:\t{accuracy_euclidean * 100:.2f}\n")

        if output_path is not None and self.write_csv:
            csv_path = os.path.join(output_path, self.csv_file)
            if not os.path.isfile(csv_path):
                with open(csv_path, newline="", mode="w", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(self.csv_headers)
                    writer.writerow([epoch, steps, accuracy_cos, accuracy_manhattan, accuracy_euclidean])

            else:
                with open(csv_path, newline="", mode="a", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow([epoch, steps, accuracy_cos, accuracy_manhattan, accuracy_euclidean])

        self.primary_metric = {
            SimilarityFunction.COSINE: "cosine_accuracy",
            SimilarityFunction.DOT_PRODUCT: "dot_accuracy",
            SimilarityFunction.EUCLIDEAN: "euclidean_accuracy",
            SimilarityFunction.MANHATTAN: "manhattan_accuracy",
        }.get(self.main_distance_function, "max_accuracy")
        metrics = {
            "cosine_accuracy": accuracy_cos,
            "dot_accuracy": accuracy_dot,
            "manhattan_accuracy": accuracy_manhattan,
            "euclidean_accuracy": accuracy_euclidean,
            "max_accuracy": max(accuracy_cos, accuracy_manhattan, accuracy_euclidean),
        }
        metrics = self.prefix_name_to_metrics(metrics, self.name)
        return metrics

In [19]:
evaluator_name = "gooaq"

In [20]:
valid_data

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 4924
})

#### Setting closed_source = True, just uses the closed source embeddings directly for evaluation, does not pass through the adapter model

In [21]:
valid_evaluator = TripletEvaluator(
        anchors=valid_data['anchor'],
        positives=valid_data['positive'],
        negatives=valid_data['negative'],
        batch_size=100,
        name=evaluator_name,
        write_csv=True
)

In [22]:
valid_evaluator(adapter_model, closed_source=True, output_path="D:/adapter_model")

TripletEvaluator: Evaluating the model on the gooaq dataset:
Accuracy Cosine Distance:   	78.01
Accuracy Dot Product:       	24.15
Accuracy Manhattan Distance:	77.52
Accuracy Euclidean Distance:	78.05



{'gooaq_cosine_accuracy': 0.7800568643379366,
 'gooaq_dot_accuracy': 0.24147034930950448,
 'gooaq_manhattan_accuracy': 0.775182778229082,
 'gooaq_euclidean_accuracy': 0.7804630381803412,
 'gooaq_max_accuracy': 0.7804630381803412}

In [23]:
valid_evaluator(adapter_model, closed_source=False, output_path="D:/adapter_model")

TripletEvaluator: Evaluating the model on the gooaq dataset:


100%|██████████| 50/50 [00:00<00:00, 68.16it/s]


Accuracy Cosine Distance:   	77.13
Accuracy Dot Product:       	24.76
Accuracy Manhattan Distance:	77.56
Accuracy Euclidean Distance:	77.62



{'gooaq_cosine_accuracy': 0.7713241267262388,
 'gooaq_dot_accuracy': 0.2475629569455727,
 'gooaq_manhattan_accuracy': 0.7755889520714866,
 'gooaq_euclidean_accuracy': 0.7761982128350934,
 'gooaq_max_accuracy': 0.7761982128350934}

In [24]:
test_evaluator = TripletEvaluator(
        anchors=test_data['anchor'],
        positives=test_data['positive'],
        negatives=test_data['negative'],
        batch_size=100,
        name=evaluator_name,
        write_csv=True
)

In [25]:
test_evaluator(adapter_model, closed_source=True, output_path="D:/adapter_model")

TripletEvaluator: Evaluating the model on the gooaq dataset:
Accuracy Cosine Distance:   	78.42
Accuracy Dot Product:       	23.69
Accuracy Manhattan Distance:	79.24
Accuracy Euclidean Distance:	79.13



{'gooaq_cosine_accuracy': 0.7841814837522992,
 'gooaq_dot_accuracy': 0.23686899652564888,
 'gooaq_manhattan_accuracy': 0.7923564275495606,
 'gooaq_euclidean_accuracy': 0.791334559574903,
 'gooaq_max_accuracy': 0.7923564275495606}

In [26]:
test_evaluator(adapter_model, closed_source=False, output_path="D:/adapter_model")

TripletEvaluator: Evaluating the model on the gooaq dataset:


100%|██████████| 49/49 [00:00<00:00, 68.17it/s]


Accuracy Cosine Distance:   	78.34
Accuracy Dot Product:       	24.24
Accuracy Manhattan Distance:	78.30
Accuracy Euclidean Distance:	78.58



{'gooaq_cosine_accuracy': 0.783363989372573,
 'gooaq_dot_accuracy': 0.24238708358880032,
 'gooaq_manhattan_accuracy': 0.78295524218271,
 'gooaq_euclidean_accuracy': 0.7858164725117515,
 'gooaq_max_accuracy': 0.7858164725117515}

In [27]:
def save_checkpoint(epoch, training_steps, model, optimizer, scheduler, filename='checkpoint.pth'):
    checkpoint = {
        'epoch': epoch,
        'training_steps': training_steps,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
    }
    torch.save(checkpoint, filename)
    print(f"Checkpoint saved at epoch {epoch}, training step {training_steps}.")

def load_checkpoint(filename, model, optimizer, scheduler):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    epoch = checkpoint['epoch']
    training_steps = checkpoint['training_steps']
    print(f"Checkpoint loaded from epoch {epoch}, training step {training_steps}.")
    return epoch, training_steps

def normalize_embeddings(embeddings: Tensor) -> Tensor:
    """
    Normalizes the embeddings matrix, so that each sentence embedding has unit length.

    Args:
        embeddings (Tensor): The input embeddings matrix.

    Returns:
        Tensor: The normalized embeddings matrix.
    """
    return torch.nn.functional.normalize(embeddings, p=2, dim=1)

def cos_sim(a: Tensor, b: Tensor) -> Tensor:
    """
    Computes the cosine similarity between two tensors.

    Args:
        a (Union[list, np.ndarray, Tensor]): The first tensor.
        b (Union[list, np.ndarray, Tensor]): The second tensor.

    Returns:
        Tensor: Matrix with res[i][j] = cos_sim(a[i], b[j])
    """

    a_norm = normalize_embeddings(a)
    b_norm = normalize_embeddings(b)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

def MultipleNegativesRankingLoss(features, scale: float = 20.0):
    cross_entropy_loss = nn.CrossEntropyLoss()
    embeddings_a = features[0]
    embeddings_b = torch.cat(features[1:])
    scores = cos_sim(embeddings_a, embeddings_b) * scale
    range_labels = torch.arange(0, scores.size(0), device=scores.device)
    return cross_entropy_loss(scores, range_labels)

In [28]:
def train_adapter_model(
        model: nn.Module,
        output_path: str,
        train_dataloader,
        epochs,
        evaluator: None,
        evaluation_steps: int = 0,
        eval_score_criteria: str = "cosine_accuracy", # One of ["cosine_accuracy", "dot_accuracy", "manhattan_accuracy", "euclidean_accuracy", "max_accuracy"]
        scheduler: str = 'WarmupLinear',
        warmup_steps: int = 10000,
        optimizer_class: Type[Optimizer] = torch.optim.AdamW,
        optimizer_params: Dict[str, object] = {'lr': 2e-5},
        weight_decay: float = 0.01,
        model_target_device: str = "cpu",
        load_from_checkpoint: bool = False,
        checkpoint_path: str = None,
        show_progress_bar: bool = True,
        save_strategy: str = "epoch", # Can be one of "epoch" or "steps"
        save_steps: int = 100, # Only applicable if save_strategy is set to "steps"
):

    valid_score_criteria = ["cosine_accuracy", "dot_accuracy", "manhattan_accuracy", "euclidean_accuracy", "max_accuracy"]

    if type(eval_score_criteria) == str:
        if eval_score_criteria not in valid_score_criteria:
            raise Exception(f"Please provide a valid value for evaluation score criteria. Must be one of {str(valid_score_criteria)} and a string value")
    else:
        raise Exception(f"Please provide a valid value for evaluation score criteria. Must be one of {str(valid_score_criteria)} and a string value")

    if load_from_checkpoint and (checkpoint_path is None or type(checkpoint_path) != str):
        raise Exception("Checkpoint Path must be a valid string when load_from_checkpoint is set to True.")

    model = model.to(model_target_device)

    if output_path is not None:
        os.makedirs(output_path, exist_ok=True)
        # Creating Checkpoint Folder
        os.makedirs(output_path + "/checkpoints", exist_ok=True)

    num_train_steps = int(len(train_dataloader) * epochs)

    # Prepare optimizers
    param_optimizer = list(model.named_parameters())

    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)

    best_score = -9999999

    if isinstance(scheduler, str):
        scheduler = SentenceTransformer._get_scheduler(optimizer, scheduler=scheduler, warmup_steps=warmup_steps, t_total=num_train_steps)

    start_epoch = 0
    start_training_steps = 0
    if load_from_checkpoint:
        start_epoch, start_training_steps = load_checkpoint(checkpoint_path, model, optimizer, scheduler)
    if start_training_steps == -1:
        start_epoch += 1
        start_training_steps = 0

    skip_scheduler = False
    for epoch in trange(start_epoch, epochs, desc="Epoch", disable=not show_progress_bar):
        training_steps = start_training_steps
        model.zero_grad()
        model.train()

        for _ in range(start_training_steps):
            next(train_dataloader) 

        for features in tqdm(train_dataloader, desc="Iteration", smoothing=0.05, disable=not show_progress_bar):
            list_closed_embeddings = []
            for k, v in features.items():
                list_closed_embeddings.append([dict_embeddings[val] for val in v])
            list_closed_embeddings_tensor = torch.tensor(list_closed_embeddings, dtype=torch.float32)
            reps = [adapter_model(embedding) for embedding in list_closed_embeddings_tensor]
            loss = MultipleNegativesRankingLoss(reps)
            loss.backward()

            optimizer.step()

            optimizer.zero_grad()

            if not skip_scheduler:
                scheduler.step()

            print(f"Epoch: {epoch}, Training Step: {training_steps}, Loss: {loss.item()}")

            if save_strategy == "steps" and save_steps is not None:
                if training_steps % save_steps == 0:
                    save_checkpoint(epoch, training_steps, model, optimizer, scheduler, output_path + f"/checkpoints/model_checkpoint_{epoch}_{training_steps}.pth")

            if evaluator is not None and evaluation_steps > 0 and training_steps % evaluation_steps == 0 and training_steps > 0:
                eval_result = evaluator(model, closed_source=False, output_path=output_path, epoch=epoch, steps=training_steps)
                if eval_result[f"{evaluator_name}_{eval_score_criteria}"] > best_score:
                    save_checkpoint(epoch, training_steps, model, optimizer, scheduler, output_path + f"/checkpoints/best_model.pth")
                    best_score = eval_result[f"{evaluator_name}_{eval_score_criteria}"]

                model.zero_grad()
                model.train()

            training_steps += 1

        if save_strategy == "epoch":
            save_checkpoint(epoch, training_steps, model, optimizer, scheduler, output_path + f"/checkpoints/model_checkpoint_{epoch}_{training_steps}.pth")

        if evaluator is not None:
            eval_result = evaluator(model, closed_source=False, output_path=output_path, epoch=epoch, steps=training_steps)
            if eval_result[f"{evaluator_name}_{eval_score_criteria}"] > best_score:
                save_checkpoint(epoch, training_steps, model, optimizer, scheduler, output_path + f"/checkpoints/best_model.pth")
                best_score = eval_result[f"{evaluator_name}_{eval_score_criteria}"]

    save_checkpoint(epoch, training_steps, model, optimizer, scheduler, output_path + f"/checkpoints/model_checkpoint_{epoch}_{training_steps}.pth")

In [29]:
train_adapter_model(
    model=adapter_model,
    output_path = "D:/adapter_model",
    train_dataloader = train_dl,
    epochs = 3,
    evaluator = valid_evaluator,
    evaluation_steps = 50,
    eval_score_criteria = "cosine_accuracy",
    scheduler = "warmupcosine",
    warmup_steps = 10,
    optimizer_class = torch.optim.AdamW,
    optimizer_params = {'lr': 2e-5},
    weight_decay = 0.01,
    model_target_device = "cpu",
    load_from_checkpoint = False,
    checkpoint_path= None,
    show_progress_bar = True,
    save_strategy = "steps", # Can be one of "epoch" or "steps"
    save_steps = 50, # Only applicable if save_strategy is set to "steps"
)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0, Training Step: 0, Loss: 0.8012745380401611
Checkpoint saved at epoch 0, training step 0.
Epoch: 0, Training Step: 1, Loss: 0.7857745289802551
Epoch: 0, Training Step: 2, Loss: 0.7289088368415833
Epoch: 0, Training Step: 3, Loss: 0.7004703283309937
Epoch: 0, Training Step: 4, Loss: 0.6883841156959534


Epoch: 0, Training Step: 5, Loss: 0.74229896068573
Epoch: 0, Training Step: 6, Loss: 0.6582251191139221
Epoch: 0, Training Step: 7, Loss: 0.5318671464920044
Epoch: 0, Training Step: 8, Loss: 0.5743264555931091


Epoch: 0, Training Step: 9, Loss: 0.5228235721588135
Epoch: 0, Training Step: 10, Loss: 0.6766355037689209
Epoch: 0, Training Step: 11, Loss: 0.6655847430229187
Epoch: 0, Training Step: 12, Loss: 0.5950328707695007
Epoch: 0, Training Step: 13, Loss: 0.5642753839492798


Epoch: 0, Training Step: 14, Loss: 0.5925382971763611
Epoch: 0, Training Step: 15, Loss: 0.7271687388420105
Epoch: 0, Training Step: 16, Loss: 0.684628963470459
Epoch: 0, Training Step: 17, Loss: 0.6661151051521301


...

Epoch: 2, Training Step: 876, Loss: 0.4513179361820221
Epoch: 2, Training Step: 877, Loss: 0.5028624534606934
Epoch: 2, Training Step: 878, Loss: 0.45588305592536926
Epoch: 2, Training Step: 879, Loss: 0.38851550221443176
Epoch: 2, Training Step: 880, Loss: 0.36979636549949646


Iteration: 884it [01:22, 10.75it/s]                         


Epoch: 2, Training Step: 881, Loss: 0.36669084429740906
Epoch: 2, Training Step: 882, Loss: 0.3996413052082062
Epoch: 2, Training Step: 883, Loss: 0.34736117720603943
TripletEvaluator: Evaluating the model on the gooaq dataset in epoch 2 after 884 steps:


Epoch: 100%|██████████| 3/3 [04:21<00:00, 87.28s/it]

Accuracy Cosine Distance:   	81.30
Accuracy Dot Product:       	19.92
Accuracy Manhattan Distance:	81.11
Accuracy Euclidean Distance:	81.19

Checkpoint saved at epoch 2, training step 884.


In [30]:
test_evaluator(adapter_model, closed_source=False, output_path="D:/adapter_model")

TripletEvaluator: Evaluating the model on the gooaq dataset:


100%|██████████| 49/49 [00:00<00:00, 80.20it/s]


Accuracy Cosine Distance:   	83.28
Accuracy Dot Product:       	17.92
Accuracy Manhattan Distance:	83.45
Accuracy Euclidean Distance:	83.51



{'gooaq_cosine_accuracy': 0.8328223993460045,
 'gooaq_dot_accuracy': 0.17923564275495607,
 'gooaq_manhattan_accuracy': 0.8344573881054568,
 'gooaq_euclidean_accuracy': 0.8350705088902514,
 'gooaq_max_accuracy': 0.8350705088902514}